In [1]:
#load packages
import json
import pymongo
from flask import Flask, request, render_template, session, redirect
from flask import request
from pandas import json_normalize
import pandas as pd

In [18]:
json_business = open('yelp_academic_dataset_business.json').readlines()
df_business = []
for line in json_business:
    df_business.append(json.loads(line))
df_business = json_normalize(df_business)    

In [20]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,attributes.AcceptsInsurance,attributes.BestNights,attributes.BYOB,attributes.Corkage,attributes.BYOBCorkage,attributes.HairSpecializesIn,attributes.Open24Hours,attributes.RestaurantsCounterService,attributes.AgesAllowed,attributes.DietaryRestrictions
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
json_review = open('yelp_academic_dataset_review.json').readlines()
df_review = []
for line in json_review:
    df_review.append(json.loads(line))
df_review = json_normalize(df_review) 

In [22]:
df_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [23]:
# import data
def import_data():
    # business data
    json_business = open('yelp_academic_dataset_business.json').readlines()
    df_business = []
    for line in json_business:
        df_business.append(json.loads(line))
    df_business = json_normalize(df_business)    
    # review data
    json_review = open('yelp_academic_dataset_review.json').readlines()
    df_review = []
    for line in json_review:
        df_review.append(json.loads(line))
    df_review = json_normalize(df_review)    
    df = pd.merge(df_business, df_review, on = 'business_id')
    return(df)

In [40]:
df = import_data()

In [41]:
df.shape

(6990280, 68)

In [10]:
df2[:20000].to_csv('df.csv')

In [ ]:
# df2 = df[:50000]
df2 = pd.read_csv('df.csv')['business_id', 'name', 'address', 'city', 'state', 'postal code', 'stars_x', 'categories', 'review']
df2 = df2.loc[df2.categories.str.contains('Restaurant', na=False)]
kw = 'pizza'

kw = kw.title() + '|' +kw.upper() + '|' +kw.lower()
df2 = df2[df2['text'].str.contains(kw)]
df2.head()

In [ ]:
app = Flask(__name__, template_folder='template')

@app.route('/', methods=("POST", "GET"))
def html_table():
    df = pd.read_csv('df.csv')
    df2 = pd.DataFrame()    
    kw = request.form.get("kw","")
    if kw:
        kw = kw.title() + '|' +kw.upper() + '|' +kw.lower()
        df2 = df[df['text'].str.contains(kw)]
        df2 = df2[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'stars_x', 'categories','text']]
        df2.rename(columns = {'stars_x' :'stars', 'text':'review'}, inplace=True)
    
    return f"""
    <html>

    <body style="text-align:center">

    <h1>Query Keywords</h1>

    <form action="" method="post">

      Key Word: <input type="text" name="kw",value="{kw}">

      <input type="submit" name="submit" value="Query">

    </form>

    <center> %s </center>

    </body>

    </html>

  """ % df2.to_html(index=False)
if __name__ == '__main__':
    app.run(host = 'localhost', port = 5004)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5004/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Aug/2022 21:32:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 21:32:59] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 21:33:07] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 21:34:13] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 21:39:28] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 21:39:33] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 21:39:36] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 21:39:53] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 21:42:15] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [06/Aug/2022 21:43:45] "POST / HTTP/1.1" 200 -
